In [ ]:
%load_ext autoreload
%autoreload 2
# Imports necessary libraries
from pathlib import Path

import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt

import suite2p

In [ ]:
# Figure Style settings for the notebook.
import matplotlib as mpl

mpl.rcParams.update(
    {
        "axes.spines.left": False,
        "axes.spines.bottom": False,
        "axes.spines.top": False,
        "axes.spines.right": False,
        "legend.frameon": False,
        "figure.subplot.wspace": 0.01,
        "figure.subplot.hspace": 0.01,
        "figure.figsize": (18, 13),
        "ytick.major.left": False,
    }
)
jet = mpl.colormaps.get_cmap("jet")
jet.set_bad(color="k")

# Visualizing Multi-Day Pipeline Outputs
This notebook contains the same data-plotting code as the 'Multi-Day Pipeline' notebook. It is designed to visualize the
data processing with the multi-day pipeline, without running the pipeline itself. Use it to evaluate the output of the
multi-day pipeline on your data.

**Note!** The single-day pipeline does not require a visualization notebook, as all single-day data can be evaluated
using the suite2p GUI. To call the GUI, use the `ss2p-gui` command from the terminal after activating the python
environment where the sl-suite2p library is installed. In the future, the multi-day visualization will also be
integrated into the suite2p GUI.

 ## Selecting the data to visualize

All functions below are designed to load all necessary data via the multi-day 'ops.npy' file. Therefore, to configure
this notebook to run on specific pipeline output, you need to provide the path to the 'ops.npy' file of the target
multi-day runtime. This file is stored under the root output folder used by that runtime, defined via the
'multiday_save_folder' field of the MultiDayS2PConfiguration class or 'ops' dictionary.

In [ ]:
ops_path = Path("/home/cyberaxolotl/Desktop/Data/11/ops.npy")

# Also loads the 'ops.npy' as a dictionary to be used by the ROI activity visualizer below
ops = np.load(ops_path, allow_pickle=True).item()

## Registration Visualization

The interactive widget below is used to visualize and assess the quality of all across-day registration steps.
It works similar to the general suite2p gui image and mask overlay visualizer, but works with the data generated
by the multi-day pipeline.

In [ ]:
%matplotlib widget
suite2p.show_images_with_masks(ops_path=ops_path);

## Cell Activity Visualization

The plotting code below visualizes the activity of a single ROI (cell) across all processed sessions. Change the target
ROI index and re-render the figure to evaluate different cells.

In [ ]:
# Change the ROI index here and re-run the cell below to visualize different ROIs
roi_index = 10

In [ ]:
# Clears the widget figure from the previous code cell
plt.close("all")

%matplotlib inline

# Precreates temporary storage lists for fluorescence data
f_cells_sessions = []
f_neuropils_sessions = []
spks_sessions = []

# Root multi-day output folder is expected to be the parent of the ops.npy file
output_folder = ops_path.parent

# The output folder contains .npy and .yaml files and directories named after each processed session ID.
# This re-generates the list of session IDs from the directories stored in the output folder.
session_ids = [folder.stem for folder in output_folder.glob("*") if folder.is_dir()]

# Sorts session IDs for consistency
session_ids = natsorted(session_ids)

# Resolves paths to the multi-day output for each session
session_directories = [output_folder.joinpath(session_id) for session_id in session_ids]

# Loops over processed sessions and loads the data to be visualized below
for session in session_directories:
    f_cells_sessions.append(np.load(session.joinpath("F.npy")))
    f_neuropils_sessions.append(np.load(session.joinpath("Fneu.npy")))
    spks_sessions.append(np.load(session.joinpath("spks.npy")))

plt.figure(figsize=(20.0, 20.0))
plt.suptitle(f"Fluorescence and Deconvolved Traces for ROI {roi_index} Across Sessions", y=0.92)

# Assigns distinct color to visualized traces
colors = ["#1f77b4", "#2ca02c", "#d62728"]  # Blue, Green, Red

# Loops over sessions, extracts the fluorescence data for the specified cell from each session, and plots it
# on the canvas.
for i, (f_cells, f_neuropils, spks, session_name) in enumerate(
    zip(f_cells_sessions, f_neuropils_sessions, spks_sessions, session_ids)
):
    plt.subplot(len(f_cells_sessions), 1, i + 1)

    # Extracts data for the specific ROI from this session
    f = f_cells[roi_index]
    f_neu = f_neuropils[roi_index]
    sp = spks[roi_index]

    # Adjust range to match fluorescence traces
    fmax = np.maximum(f.max(), f_neu.max())
    fmin = np.minimum(f.min(), f_neu.min())
    frange = fmax - fmin

    # Normalizes spikes
    if sp.max() > 0:
        sp = sp / sp.max() * frange + fmin
    else:
        sp = np.zeros_like(sp) + fmin

    plt.plot(f, color=colors[0], label="Cell Fluorescence")
    plt.plot(f_neu, color=colors[1], label="Neuropil Fluorescence")
    plt.plot(sp, color=colors[2], label="Deconvolved")

    plt.xticks(np.arange(0, f.shape[0], f.shape[0] // 10))

    # Add title for session name above each plot
    plt.title(f"Session {session_name}")

    # Add y-axis label for fluorescence/pixel intensity
    plt.ylabel("fluorescence")

    plt.xlabel("frame")
    plt.grid(True, linestyle=":", alpha=0.6)

    if i == 0:
        plt.legend(bbox_to_anchor=(1.01, 1), loc="upper left")

plt.tight_layout()
plt.subplots_adjust(top=0.9)